In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

In [1]:


def get_driver():
    chrome_service = Service('/opt/homebrew/bin/chromedriver')  # 실제 경로로 업데이트
    chrome_options = Options()
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    driver.implicitly_wait(3)
    return driver

def crawl_page(driver, page_url):
    driver.get(page_url)
    time.sleep(2)

    # '여행코스' 클릭
    driver.find_element(By.XPATH, '//*[@id="t_course"]').click()
    time.sleep(2)

    details = []

    # 각 상세 페이지 링크 크롤링
    for i in range(1, 11):  # 첫 번째 페이지에 최대 10개
        try:
            link_xpath = f'//*[@id="search_result"]/ul/li[{i}]/div[1]/div[1]/a'
            driver.find_element(By.XPATH, link_xpath).click()
            time.sleep(2)

            # JavaScript 실행하여 h2 요소에서 em 요소를 제외한 텍스트 가져오기
            script = '''
                var h2Element = document.querySelector('#contents > div:nth-child(1) > div:nth-child(2) > h2');
                var emElement = h2Element.querySelector('em');
                if (emElement) emElement.remove();
                return h2Element.innerText;
            '''
            title = driver.execute_script(script)

            element = driver.find_element(By.XPATH, '//*[@id="dist"]')
            distance = element.text

            element = driver.find_element(By.XPATH, '//*[@id="contents"]/div[2]/div[2]/div/div[1]/div/p')
            explain = element.text

            details.append({
                'title': title,
                'distance': distance,
                'explain': explain
            })

            # "더보기" 버튼이 있는 경우 클릭하여 모든 코스를 로드
            while True:
                try:
                    more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.swiper-button-next')))
                    more_button.click()
                    time.sleep(2)
                except:
                    break

            # 코스 리스트 가져오기
            course_list = driver.find_elements(By.CSS_SELECTOR, 'ul.cosList > li')
            processed_courses = set()

            for index, course in enumerate(course_list, start=1):
                course_link = course.find_element(By.TAG_NAME, 'a')
                tab_id = course_link.get_attribute('href').split('#')[-1]

                if tab_id in processed_courses:
                    continue

                driver.execute_script("arguments[0].click();", course_link)
                time.sleep(2)

                element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="{tab_id}"]/div/div[1]/div/em')))
                order = element.text
                element = driver.find_element(By.XPATH, f'//*[@id="{tab_id}"]/div/div[1]/div/strong/a')
                destination = element.text
                element = driver.find_element(By.XPATH, f'//*[@id="{tab_id}"]/div/div[1]/span[1]')
                address = element.text
                element = driver.find_element(By.XPATH, f'//*[@id="{tab_id}"]/div/ul')
                tag = element.text

                details.append({
                    'order': order,
                    'destination': destination,
                    'address': address,
                    'tag': tag
                })

                processed_courses.add(tab_id)
            
            driver.back()  # 이전 페이지로 돌아가기
            time.sleep(2)
        except Exception as e:
            print(f"Error crawling link {i}: {e}")
            break

    return details

def crawl_all_pages(base_url, max_pages=5):
    driver = get_driver()
    all_details = []
    page_num = 1

    while page_num <= max_pages:
        page_url = f'{base_url}&page={page_num}'
        print(f'Crawling page: {page_num}')
        details = crawl_page(driver, page_url)
        all_details.extend(details)
        page_num += 1
        time.sleep(2)

    driver.quit()
    return all_details

base_url = 'https://korean.visitkorea.or.kr/search/search_list.do?keyword=%EB%93%9C%EB%9D%BC%EC%9D%B4%EB%B8%8C'
all_details = crawl_all_pages(base_url, max_pages=5)

for detail in all_details:
    print(detail)


Crawling page: 1
Error crawling link 2: Message: javascript error: Cannot read properties of null (reading 'querySelector')
  (Session info: chrome=125.0.6422.142)
Stacktrace:
0   chromedriver                        0x00000001013024c8 chromedriver + 4302024
1   chromedriver                        0x00000001012fae10 chromedriver + 4271632
2   chromedriver                        0x0000000100f2c19c chromedriver + 278940
3   chromedriver                        0x0000000100f3066c chromedriver + 296556
4   chromedriver                        0x0000000100f32740 chromedriver + 304960
5   chromedriver                        0x0000000100fa772c chromedriver + 784172
6   chromedriver                        0x0000000100fa6c5c chromedriver + 781404
7   chromedriver                        0x0000000100f63004 chromedriver + 503812
8   chromedriver                        0x0000000100f639ec chromedriver + 506348
9   chromedriver                        0x00000001012ca510 chromedriver + 4072720
10  chromed